In [1]:
import numpy as np
import igl
import meshplot as mp

In [2]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [3]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

In [4]:
def find_closed_point(point, points):
    distance=np.linalg.norm(points-point,axis=1)
    return np.argmin(distance)


eps=igl.bounding_box_diagonal(pi)*0.01
piplus=np.zeros_like(pi)
piminus=np.zeros_like(pi)
for (i,point) in enumerate(pi):
    temp=point+eps*ni[i]
    neweps=eps
    while find_closed_point(temp,pi)!=i:
        neweps=neweps/2
        temp=point+neweps*ni[i]
    piplus[i]=temp
for (i,point) in enumerate(pi):
    temp=point-eps*ni[i]
    neweps=eps
    while find_closed_point(temp,pi)!=i:
        neweps=neweps/2
        temp=point-neweps*ni[i]
    piminus[i]=temp


p=mp.plot(pi,c=np.ones_like(pi)*np.array([0, 0, 1]), shading={"point_size": 8})
p.add_points(piplus,c=np.ones_like(pi)*np.array([1, 0, 0]), shading={"point_size": 8})
p.add_points(piminus,c=np.ones_like(pi)*np.array([0, 1, 0]), shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

2

# MLS function

In [37]:
# Parameters
bbox_min = np.array([-1., -1., -1.])
bbox_max = np.array([1., 1., 1.])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)
n = 128

bv,bf=igl.bounding_box(pi)
#print(bv)
#print(bf)
#print(list(np.min(np.array(bv),axis=0)))
V,F=tet_grid([n,n,n],np.min(np.array(bv),axis=0)*1.1,np.max(np.array(bv),axis=0)*1.1)
#p.add_points(V,c=np.ones_like(V)*np.array([0.5, 0.5, 0.5]), shading={"point_size": 2})


In [38]:
def normalize(points,min,max):
    return ((points-np.min(points,axis=0))/(np.max(np.max(points,axis=0)-np.min(points,axis=0))))*(max-min)-(max-min)/2
npi=normalize(pi,-1,1)
npiplus=normalize(piplus,-1,1)
npiminus=normalize(piminus,-1,1)
#print(npi)
#mp.plot(npi,v)

In [39]:
# Generate grid n x n x n
x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)
#x,T=tet_grid((n, n, n), bbox_min, bbox_max)
#npi=npi+np.array([-1,-1,0])
p=mp.plot(npi,v)
p.add_points(x,shading={"point_size": 0.1})
#print(npi)
#print(np.min(npi))
#print(np.max(npi))
#print(x)
#print(np.min(x))
#print(np.max(x))



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.471374…

1

In [43]:
wendlandRadius=0.3
coefficientnumber=10
def closest_points(point, points, h):
    distance=np.linalg.norm(points-point,axis=1)
    return np.argwhere(distance < h)

def wendlandweight(p1,p2,h):
    distance=np.linalg.norm(p1-p2)
    return (1-distance/h)**4 * (4*distance/h+1)

def functioninvectork2(x,y,z):
    return np.array([1,x,y,z,x**2,x*y,x*z,y**2,y*z,z**2])
funcargs=np.zeros((x.shape[0],coefficientnumber)) #1,x,y,z,x^2,xy,xz,y^2,yz,z^2
points=np.concatenate((npi,npiplus,npiminus))
fx=np.zeros((x.shape[0],1))
for (i,xi) in enumerate(x): 
    adjlist=np.array(closest_points(xi,points,wendlandRadius)).squeeze(axis=1)
    a=np.zeros((adjlist.size,coefficientnumber))
    b=np.zeros((adjlist.size,1))
    w=np.zeros((adjlist.size,adjlist.size)) #diagonal matrix
    #print(adjlist.shape) if adjlist.shape[0]!=0 else None
    if(len(adjlist)<coefficientnumber*2):
        fx[i]=1000
        continue
    #print(adjlist)
    for (j,adj) in enumerate(adjlist):
        adj_position=points[adj]
        originalpiindex=adj % (npi.shape[0])
        originalpi_position=points[originalpiindex]
        eps=(originalpi_position-adj_position)[0]/ni[originalpiindex,0]
        a[j,:]=functioninvectork2(adj_position[0],adj_position[1],adj_position[2])
        b[j]=eps  #what would be range?
        w[j,j]=wendlandweight(adj_position,xi,wendlandRadius)
        #print(a[j,:])
        #print(b[j])
        #print(w[j,j])
    funcargs[i]=np.linalg.solve(a.T@w@a,a.T@w@b).T
    fx[i]=(functioninvectork2(xi[0],xi[1],xi[2])@funcargs[i]).T
    
    

In [44]:
# Treshold fx to visualize inside outside

ind = np.zeros_like(fx)
#print(fx.shape)
#print(ind.shape)
ind[fx >= 0] = 1
ind[fx < 0] = -1
#print(ind)
mp.plot(x, c=ind, shading={"point_size": 0.1,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Marching to extract surface

In [45]:
# Marcing tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.461889…